In [1]:
import torch
import numpy as np
import gym
from tqdm import tqdm
import time
from collections import namedtuple

from function_approximators.function_approximators import NeuralNetwork, LinearModel

from agents.agents import DQNAgent, LinearAgent

env = gym.make("CartPole-v1")

In [24]:
def _tiling_features(x):
    
    tiling_coords_theta = np.array([-0.18,-0.13,-0.08,-0.03,0.07,0.12,0.17,0.22])
    tiling_coords_pos = np.array([-3.2,-1.2,-0.7,0.3,0.8,2.8])

    a = []

    try:
        for i in range(3):
            f_tiling_theta = np.digitize(x[:,2], tiling_coords_theta+i*0.02)
            
            new_features_theta = np.zeros((x.shape[0],len(tiling_coords_theta)+1))
            new_features_theta[np.arange(x.shape[0]),f_tiling_theta]=1
            a.append(torch.Tensor(new_features_theta))
            
            f_tiling_pos = np.digitize(x[:,0], tiling_coords_pos+i*0.2)
            
            new_features_pos = np.zeros((x.shape[0],len(tiling_coords_pos)+1))
            new_features_pos[np.arange(x.shape[0]),f_tiling_pos]=1
            a.append(torch.Tensor(new_features_pos))
        
        return torch.cat([x[:,[1,3]],a[0],a[1],a[2],a[3],a[4],a[5]], -1)

    except:
        for i in range(3):
            f_tiling_theta = np.digitize(x[2], tiling_coords_theta+i*0.02)
            
            new_features_theta = np.zeros(len(tiling_coords_theta)+1)
            new_features_theta[f_tiling_theta]=1
            a.append(torch.Tensor(new_features_theta))
            
            f_tiling_pos = np.digitize(x[0], tiling_coords_pos+i*0.2)
            
            new_features_pos = np.zeros(len(tiling_coords_pos)+1)
            new_features_pos[f_tiling_pos]=1
            a.append(torch.Tensor(new_features_pos))
        
        return torch.cat([x[[1,3]],a[0],a[1],a[2],a[3],a[4],a[5]], -1)

In [25]:
x = torch.Tensor([1,2,3,4])
print(_tiling_features(x))

IndexError: too many indices for tensor of dimension 1

In [27]:
x[[1,3]]

tensor([2., 4.])